In [1]:
from sklearn.cluster import DBSCAN
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import pandas as pd
from open3d import visualization, io
import matplotlib.pyplot as plt
import ast
import os
import time
import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
Path_pcds_processed = "./pcds_processed"
view_config = "saved_view.json"

In [3]:
vis= o3d.visualization.Visualizer()
vis.create_window(width= 1440, height= 900)

point_cloud = io.read_point_cloud(os.path.join(Path_pcds_processed, "point_cloud_{:0>3}.pcd".format(0))) # Read first point cloud
vis.add_geometry(point_cloud)
vis.update_renderer()

try:
    with open(view_config, 'r') as f:
        view_params = json.load(f)
    
    # PinholeCameraParameters 
    ctr = vis.get_view_control()
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    
    # extrinsic 
    extrinsic = np.array(view_params['extrinsic']).reshape((4, 4))
    camera_params.extrinsic = extrinsic
    
    # intrinsic 
    intrinsic = o3d.camera.PinholeCameraIntrinsic()
    intrinsic.intrinsic_matrix = np.array(view_params["intrinsic"]['intrinsic_matrix']).reshape((3, 3))
    intrinsic.width = view_params["intrinsic"]['width']
    intrinsic.height = view_params["intrinsic"]['height']
    camera_params.intrinsic = intrinsic
    
    # Apply
    ctr.convert_from_pinhole_camera_parameters(camera_params, allow_arbitrary= True)
    ctr.set_zoom(0.5)
    ctr.set_lookat([-1, 0, 0])
    
    print("View parameters loaded and applied.")
except Exception as e:
    print(f"Failed to load view parameters: {e}")


for num_point_cloud in range(1, 54):
    temo_point_cloud = io.read_point_cloud(os.path.join(Path_pcds_processed, "point_cloud_{:0>3}.pcd".format(num_point_cloud))) # Read point cloud
    point_cloud.points = temo_point_cloud.points
    point_cloud.colors = temo_point_cloud.colors
    vis.update_geometry(point_cloud)
    
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)  # 延迟时间，控制播放速度

vis.destroy_window()

View parameters loaded and applied.
